In [1]:
from __future__ import division

from staintools import ReinhardNormalizer
from staintools import MacenkoNormalizer
from staintools import VahadaneNormalizer

from staintools import standardize_brightness
from staintools.utils.visual import read_image, show, show_colors, build_stack, patch_grid

import os
import cv2
import time
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor, as_completed, wait

In [2]:
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

In [3]:
def process_image(normalizer, image_in, save_dir, depth):
    tokens = image_in.rsplit(os.sep, depth+1)
    image_out = os.path.join(save_dir, *tokens[1:])
    parent_dir = os.path.dirname(image_out)
    os.makedirs(parent_dir, exist_ok=True)
    r_img = read_image(image_in)
    target = normalizer.transform(r_img)
    target = cv2.cvtColor(target, cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_out, target)
    
def batch_process_image(images_in, save_dir, depth):
    normalizer = VahadaneNormalizer()
    i1 = read_image("./lbp_pic/reference_pic/zs_abnormal.png")
    normalizer.fit(i1)
    
    for image_in in images_in:
        process_image(normalizer, image_in, save_dir, depth)

In [4]:
class Worker:
    def __init__(self, nb_workers):
        self.executor = ProcessPoolExecutor(nb_workers)
        self.futures = []
        self.max_queue = nb_workers * 3
        self.stoped = False

#     def start_procs(self):
#         for i in range(200000):
#             while not self.stoped:
#                 if len(self.futures) <= self.max_queue:
#                     t = self.executor.submit(something_fancy)
#                     t.add_done_callback(self.done)
#                     self.futures.append(t)
#                     break
    
    def add_process(self, *args):
        while not self.stoped:
            if len(self.futures) <= self.max_queue:
                t = self.executor.submit(*args)
#                 t.add_done_callback(self.done)
                self.futures.append(t)
                break

    def done(self, future):
        print("finished. result of calling process:", future.result())
        self.futures.remove(future)

    def clear(self):
        self.stoped = True
#         for future in as_completed(self.futures):
#             print("process cleared:", future.result())
        wait(self.futures)
        print("processes cleared")
    
    def start(self):
        self.stoped = False
        self.futures = []

In [5]:
def main(input_dir, output_dir, depth):
    images_in = scan_files(input_dir, postfix=".jpg")
    print("total of {} images to process".format(len(images_in)))
    
    worker = Worker(cpu_count()//2)
    
    batch_size = 2
    queue_size = 0
    for i in range(0, len(images_in), batch_size):
        batch = images_in[i : i+batch_size].copy()
        worker.add_process(batch_process_image, batch, output_dir, depth)
        print("added {} - {} to process pool".format(i, i+batch_size))
        queue_size += 1
        if queue_size == 10:
            worker.clear()
            worker.start()
            queue_size = 0
    
    worker.clear()

In [6]:
# def main(input_dir, output_dir, depth):
#     images_in = scan_files(input_dir, postfix=".jpg")
#     print("total of {} images to process".format(len(images_in)))
    
#     executor = ProcessPoolExecutor(max_workers=cpu_count()-4)
#     tasks = []
    
#     batch_size = 1000
#     for i in range(0, len(images_in), batch_size):
#         batch = images_in[i : i+batch_size]
# #         batch_process_image(images_in=batch, save_dir=output_dir, depth=depth)
#         tasks.append(executor.submit(batch_process_image, batch, output_dir, depth))
#         print("added {} - {} to process pool".format(i, i+batch_size))
    
#     job_count = len(tasks)
#     for future in as_completed(tasks):
#         job_count -= 1
#         print("processed {} images, remaining job count: {}".format(interval, job_count))

In [7]:
input_dir = "/home/nvme/ext_299"
output_dir = "/home/nvme/ext_299_stained"
depth = 1

In [8]:
main(input_dir, output_dir, depth)

total of 36674 images to process
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
added 0 - 2 to process pool
added 2 - 4 to process pool
added 4 - 6 to process pool
added 6 - 8 to process pool
added 8 - 10 to process pool
added 10 - 12 to process pool
added 12 - 14 to process pool
added 14 - 16 to process pool
added 16 - 18 to process pool
added 18 - 20 to process pool
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 20 - 22 to process pool
added 22 - 24 to process pool
added 24 - 26 to process pool
added 26 - 28 to process pool
added 28 -

Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 260 - 262 to process pool
added 262 - 264 to process pool
added 264 - 266 to process pool
added 266 - 268 to process pool
added 268 - 270 to process pool
added 270 - 272 to process pool
added 272 - 274 to process pool
added 274 - 276 to process pool
added 276 - 278 to process pool
added 278 - 280 to process pool
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
pr

Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 500 - 502 to process pool
added 502 - 504 to process pool
added 504 - 506 to process pool
added 506 - 508 to process pool
added 508 - 510 to process pool
added 510 - 512 to process pool
added 512 - 514 to process pool
added 514 - 516 to process pool
added 516 - 518 to process pool
added 518 - 520 to process pool
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 520 - 522 to process pool
added 522 - 524 to process pool
added 524 - 526 to process pool
added 526 - 528 to process pool
added 528 - 530 to process pool
added 530 - 532 to pro

Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 760 - 762 to process pool
added 762 - 764 to process pool
added 764 - 766 to process pool
added 766 - 768 to process pool
added 768 - 770 to process pool
added 770 - 772 to process pool
added 772 - 774 to process pool
added 774 - 776 to process pool
added 776 - 778 to process pool
added 778 - 780 to process pool
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
pr

Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 1000 - 1002 to process pool
added 1002 - 1004 to process pool
added 1004 - 1006 to process pool
added 1006 - 1008 to process pool
added 1008 - 1010 to process pool
added 1010 - 1012 to process pool
added 1012 - 1014 to process pool
added 1014 - 1016 to process pool
added 1016 - 1018 to process pool
added 1018 - 1020 to process pool
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 1020 - 1022 to process pool
added 1022 - 1024 to process pool
added 1024 - 1026 to process pool
added 1026 - 1028 to process pool
added 1028 - 1030 to proce

...

Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 2680 - 2682 to process pool
added 2682 - 2684 to process pool
added 2684 - 2686 to process pool
added 2686 - 2688 to process pool
added 2688 - 2690 to process pool
added 2690 - 2692 to process pool
added 2692 - 2694 to process pool
added 2694 - 2696 to process pool
added 2696 - 2698 to process pool
added 2698 - 2700 to process pool
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightnes

Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 2920 - 2922 to process pool
added 2922 - 2924 to process pool
added 2924 - 2926 to process pool
added 2926 - 2928 to process pool
added 2928 - 2930 to process pool
added 2930 - 2932 to process pool
added 2932 - 2934 to process pool
added 2934 - 2936 to process pool
added 2936 - 2938 to process pool
added 2938 - 2940 to process pool
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightnes

Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
processes cleared
added 3160 - 3162 to process pool
added 3162 - 3164 to process pool
added 3164 - 3166 to process pool
added 3166 - 3168 to process pool
added 3168 - 3170 to process pool
added 3170 - 3172 to process pool
added 3172 - 3174 to process pool
added 3174 - 3176 to process pool
added 3176 - 3178 to process pool
added 3178 - 3180 to process pool
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightness standardization
Using brightnes

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/lukawa/.conda/envs/algo-work/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/lukawa/.conda/envs/algo-work/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lukawa/.conda/envs/algo-work/lib/python3.5/concurrent/futures/process.py", line 295, in _queue_management_worker
    shutdown_worker()
  File "/home/lukawa/.conda/envs/algo-work/lib/python3.5/concurrent/futures/process.py", line 253, in shutdown_worker
    call_queue.put_nowait(None)
  File "/home/lukawa/.conda/envs/algo-work/lib/python3.5/multiprocessing/queues.py", line 129, in put_nowait
    return self.put(obj, False)
  File "/home/lukawa/.conda/envs/algo-work/lib/python3.5/multiprocessing/queues.py", line 83, in put
    raise Full
queue.Full



BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore